In [ ]:
%pip install newsapi
%pip install newsapi-python
%pip install kafka-python
%pip install transformers torch

In [ ]:
from newsapi import NewsApiClient
import json
from kafka import KafkaProducer
from transformers import pipeline

In [ ]:
key = "93e226e768c04c51853813181c122477"

# Initialize api endpoint
newsapi = NewsApiClient(api_key=key)

In [ ]:
sources = 'bbc-news,cnn,fox-news,nbc-news,the-guardian-uk,the-new-york-times,the-washington-post,usa-today,independent,daily-mail'


In [ ]:
all_articles = newsapi.get_everything(q='france',
                                      sources=sources,
                                      language='en')

In [ ]:
def preprocess_data(article, category):
        return {
            'source': article.get('source', {}).get('name'),
            'category':category,
            'title': article.get('title'),
            'author': article.get('author'),
            'url': article.get('url'),
            'urlToImage': article.get('urlToImage'),
            'content': article.get('content')
        }
    

In [ ]:
processed_data = []
classifier = pipeline("text-classification", model="classla/multilingual-IPTC-news-topic-classifier", device=0, max_length=512, truncation=True)
for article in all_articles['articles']:
    if article['title'] != '[Removed]':
        category = classifier(article['title'])
        processed_data.append(preprocess_data(article, category[0].get('label')))

In [ ]:
for record in processed_data:
    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    producer.send('news-from-api', json.dumps(record).encode('utf-8'))
# producer.flush()
# producer.close()